In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import os
import cv2

In [ ]:
import os
img_size = 128
training_data=[]
categories=['hem','all']
folders = ['fold_0','fold_1','fold_2']
s = 0
for folder in folders:
  for category in categories:
    pt='../input/leukemia-classification/C-NMC_Leukemia/training_data'+'/'+folder
    p = os.path.join(pt,category)
    class_num = categories.index(category)
    for img in os.listdir(p):
      print(s)
      try:
        img_array = cv2.resize(cv2.imread(os.path.join(p,img)),(img_size,img_size))
        training_data.append([img_array,class_num])
        s+=1
      except Exception as e:
        print("Can't read image")


In [ ]:
from tqdm import tqdm
# Viewing Image - After Shuffle 

fig, ax = plt.subplots(nrows = 1, ncols = 5, figsize = (20,20))

for i in tqdm(range(0, 5)):
    rand = np.random.randint(len(training_data))
    ax[i].imshow(training_data[rand][0])
    ax[i].axis('off')
    a = training_data[rand][1]
    if a == 1:
        ax[i].set_title('Diseased')
    else:
        ax[i].set_title('Non_Diseased')

In [ ]:
import random
random.shuffle(training_data)

In [ ]:
print(len(training_data))

In [ ]:
x_train=[]
y_train=[]
for x,y in training_data:
    x_train.append(x)
    y_train.append(y)

In [ ]:
y_train=np.array(y_train) 
x_train=np.array(x_train)
x_train.shape

In [ ]:
from tensorflow.keras.layers import Input,Conv2D,Dense,Flatten
from tensorflow.keras.models import Model
import tensorflow as tf

In [ ]:
i=Input(shape=x_train[0].shape)
print(i.shape)
x=Conv2D(32,(5,5),strides=2,activation='relu')(i)
print(x.shape)
x=Conv2D(64,(5,5),strides=2,activation='relu')(x)
print(x.shape)
x=Flatten()(x)
print(x.shape)
x=Dense(2500,activation='relu')(x)
print(x.shape)
x=Dense(1000,activation='relu')(x)
print(x.shape)
x=Dense(500,activation='relu')(x)
print(x.shape)
x=Dense(2,activation='softmax')(x)
print(x.shape)
model=Model(i,x)
model.compile(optimizer='adam',loss='categorical_crossentropy',metrics=['accuracy'])

In [ ]:
y_train=tf.keras.utils.to_categorical(y_train,num_classes=2)

In [ ]:
r=model.fit(x=x_train,y=y_train,epochs=20,shuffle=True,batch_size=100)

In [ ]:
DATADIR_1="../input/leukemia-classification/C-NMC_Leukemia/validation_data/C-NMC_test_prelim_phase_data"
valid_data=[]
img_size=128

In [ ]:
def create_validation_data():
    i = 0
    for img in os.listdir(DATADIR_1):
        print(i)
        i += 1
        try:
            img_array = cv2.imread(os.path.join(DATADIR_1,img))
            new_array = cv2.resize(img_array,(img_size,img_size))
            valid_data.append([new_array])
        except Exception as e:
                pass

In [ ]:
create_validation_data()

In [ ]:
x_valid=np.array(valid_data).reshape(1867,128,128,3)
x_valid.shape

In [ ]:
import pandas as pd
df = pd.read_csv("../input/leukemia-classification/C-NMC_Leukemia/validation_data/C-NMC_test_prelim_phase_data_labels.csv")

In [ ]:
y_valid = df['labels']
y_valid = np.array(y_valid)

In [ ]:
y_valid=tf.keras.utils.to_categorical(y_valid,num_classes=2)

In [ ]:
model.evaluate(x=x_valid,y=y_valid,batch_size=100)

In [ ]:
data_generator = tf.keras.preprocessing.image.ImageDataGenerator(width_shift_range= 0.2, height_shift_range= 0.2,
    rotation_range= 90,horizontal_flip= True, vertical_flip=True)
train_generator = data_generator.flow(x=x_train, y=y_train, batch_size=32, shuffle=True, seed=42)
steps_per_epoch = x_train.shape[0] // 32
r1 = model.fit(train_generator, steps_per_epoch=steps_per_epoch, epochs=5,validation_data=(x_valid,y_valid))

In [ ]:
from tensorflow.keras.layers import MaxPooling2D, BatchNormalization, GlobalMaxPooling2D, Dropout
i=Input(shape=x_train[0].shape)
print(i.shape)
x=Conv2D(32,(5,5),activation='relu',padding='same')(i)
print(x.shape)
x = BatchNormalization()(x)
x=Conv2D(64,(5,5),activation='relu',padding='same')(x)
print(x.shape)
x = BatchNormalization()(x)
x = MaxPooling2D((2, 2))(x)
x = Dropout(0.2)(x)
x = GlobalMaxPooling2D()(x)
print(x.shape)
x=Dropout(0.2)(x)
x=Dense(2500,activation='relu')(x)
print(x.shape)
x=Dense(1000,activation='relu')(x)
print(x.shape)
x=Dense(500,activation='relu')(x)
print(x.shape)
x=Dropout(0.2)(x)
x=Dense(2,activation='sigmoid')(x)
print(x.shape)
model1=Model(i,x)
model1.compile(optimizer='adam',loss='binary_crossentropy',metrics=['accuracy'])

In [ ]:
r2 = model1.fit(train_generator, steps_per_epoch=steps_per_epoch, epochs=5,validation_data=(x_valid,y_valid))